In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import fastai
from fastai import * 
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *

import pandas as pd
import numpy as np
import os

fastai.version.__version__

'1.0.39'

In [3]:
# make sure CUDA is available and enabled
print('CUDA enabled:',torch.cuda.is_available()) 
print('CUDNN enabled:', torch.backends.cudnn.enabled)

CUDA enabled: True
CUDNN enabled: True


In [4]:
def recreate_directory(directory):
    !rm -R {directory} 2>nul
    !mkdir {directory}

# Dataset preprocessing

In [5]:
current_dir = os.getcwd()
input_path =f'{current_dir}/data'
train_dir = f"{input_path}/train"
train_labels = f"{input_path}/train.csv"
test_dir = f"{input_path}/test"
model_dir = f'{current_dir}/models'

## Train model

In [6]:
SZ = 224
BS = 64
NUM_WORKERS = 12
SEED=0
arch = models.resnet50

In [7]:
df = pd.read_csv(train_labels)
# df = df.sample(frac=0.25)
# df = df.reset_index()

print(df.shape)
print(df.head())

(25361, 2)
           Image         Id
0  0000e88ab.jpg  w_f48451c
1  0001f9222.jpg  w_c3d896a
2  00029d126.jpg  w_20df2c5
3  00050a15a.jpg  new_whale
4  0005c1ef8.jpg  new_whale


In [8]:
# grouped_df = df.groupby('Id')
# grouped_counted = grouped_df.count().sort_values(by=['Image'], ascending=False)
# grouped_counted = grouped_counted[(grouped_counted['Image']>5) & (grouped_counted['Image']<1000)]
# print(len(grouped_counted))
# print(grouped_counted.sum().Image, 'of', len(df))
# print(grouped_counted.head())

In [9]:
# valid_pct = 0.2

# valid_filenames = pd.DataFrame(columns=df.columns)

# for name, group in enumerate(grouped_df):
#     sub_df = group[1]
#     sample = sub_df.sample(frac=valid_pct)
#     valid_filenames = valid_filenames.append(sample, ignore_index=True)

In [10]:
# valid_filenames.drop(labels=['index'], axis=1, inplace=True, errors='ignore')

# print(valid_filenames.shape)
# print(valid_filenames.head())

In [11]:
fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [12]:
# valid_files = ItemList.from_df(df=valid_filenames, path=train_dir, cols=['Image'])

In [13]:
test_files = ImageItemList.from_folder(test_dir)

In [14]:
# TODO label from df?
# data = (
#     ImageItemList
#         .from_df(df, train_dir, cols=['Image'])
# #         .no_split()
#         .split_by_files(valid_files)
#         .label_from_func(lambda path: fn2label[path2fn(path)])
#         .add_test(test_files)
#         .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
#         .databunch(bs=BS, num_workers=NUM_WORKERS, path=input_path)
#         .normalize(imagenet_stats)
# )

In [15]:
#data.show_batch(rows=3, fig_size=(SZ, SZ))

# Learning rate

In [16]:
# data_lr = (
#     ImageItemList
#         .from_df(df, train_dir, cols=['Image'])
# #         .no_split()
#         .split_by_files(valid_files)
#         .label_from_func(lambda path: fn2label[path2fn(path)])
#         .add_test(test_files)
#         .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
#         .databunch(bs=BS, num_workers=NUM_WORKERS, path=input_path)
#         .normalize(imagenet_stats)
# )

In [17]:
# learn = create_cnn(data_lr, arch, metrics=accuracy, model_dir=model_dir)

In [18]:
# learn.lr_find()

In [19]:
# learn.recorder.plot()

# Precompute

In [20]:
data = (
    ImageItemList
        .from_df(df, train_dir, cols=['Image'])
        .no_split()
#         .split_by_files(valid_files)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(test_files)
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=input_path)
        .normalize(imagenet_stats)
)

In [21]:
learn = create_cnn(data, arch, metrics=accuracy, model_dir=model_dir, callback_fns=ShowGraph)

In [22]:
learn.fit_one_cycle(6, wd=1e-4, max_lr=1e-2)

In [23]:
learn.unfreeze()
lr_rate = 1e-3
learn.fit_one_cycle(8, wd=1e-4, max_lr=[lr_rate/100, lr_rate/10, lr_rate])

# Prediction & Summition - TODO

In [24]:
log_preds,y = learn.TTA(ds_type=DatasetType.Test)

In [25]:
def top_preds(preds, classes):
    top = np.argsort(preds.numpy())[:, ::-1][:, :5]
    labels = []
    for i in range(top.shape[0]):
        labels.append(' '.join([classes[idx] for idx in top[i]]))
    return labels

def create_submission(preds, data):
    classes = data.classes
    sub_df = pd.DataFrame({'Image': [path.name for path in data.test_ds.x.items]})
    sub_df['Id'] = top_preds(preds, classes)
    return sub_df

In [26]:
submittion_df = create_submission(log_preds, learn.data)

In [27]:
submittion_file = 'submission-4-with-restarts.csv'
submittion_df.to_csv(submittion_file, index=False)

In [28]:
print(submittion_df.head())
print(submittion_df.shape)

           Image                                                 Id
0  a82ba2f76.jpg  w_89f6097 new_whale w_6aab519 w_0531ad5 w_8e88f4f
1  ab62db5a5.jpg  w_fdcaaa8 new_whale w_34d7623 w_6f7611c w_8f1069a
2  6219200df.jpg  new_whale w_e096f02 w_4f9c015 w_e4a9205 w_0b10e7c
3  0f3c1f2ca.jpg  new_whale w_d10d0c5 w_396c12b w_1a70f3a w_7d44896
4  7d139670e.jpg  new_whale w_f602022 w_0531ad5 w_343f088 w_a6067a9
(7960, 2)


In [29]:
!kaggle competitions submit -c humpback-whale-identification -f {submittion_file} -m "learning with restarts"

100%|█████████████████████████████████████████| 498k/498k [00:02<00:00, 191kB/s]
Successfully submitted to Humpback Whale Identification